In [1]:
import pandas as pd
import numpy as np
path = r"C:\Users\kpras\Desktop\ClientsCalculatedTAT_13-01-2026 23_22_06_Tata_Motors_Finance_Limited_All.xlsx"
df = pd.read_excel(path)
print(df.head())

   SrNo                       Client                   Product ApplicationNo  \
0     1  Tata Motors Finance Limited   Tata Motors_CV_SCV_FTU1  R26925922165   
1     2  Tata Motors Finance Limited   Tata Motors_CV_SCV_FTU1  R26925922165   
2     3  Tata Motors Finance Limited   Tata Motors_CV_SCV_FTU1  R26925922165   
3     4  Tata Motors Finance Limited  Tata Motors_CV_MHCV_FTU1  R27925924803   
4     5  Tata Motors Finance Limited   Tata Motors_CV_SCV_FTU2  R28025925378   

  Application Status         Case In Time      DE Completed At  \
0            Sampled  Dec 19 2025  3:34PM  Dec 19 2025  3:29PM   
1            Sampled  Dec 19 2025  3:34PM  Dec 17 2025  5:14PM   
2            Sampled  Dec 19 2025  3:34PM  Sep 27 2025  5:01PM   
3           Screened  Dec  5 2025  3:03PM  Oct  9 2025 12:23PM   
4            Sampled  Dec  4 2025 12:57PM  Nov 28 2025 11:33AM   

                DE Completed By        Case Out Time           Processed By  \
0           Abhijeet Nachvankar  Dec 19 202

In [16]:
df_new = df[['Product', 'Application Status']].copy()
df_new['cnt'] = 1

product_sampling = pd.pivot_table(
    df_new,
    index='Product',
    columns='Application Status',
    values='cnt',
    aggfunc='sum',
    fill_value=0,
    margins=True,              # adds Grand Total
    margins_name='Grand Total'
)

print(product_sampling)


Application Status         Sampled  Screened  Grand Total
Product                                                  
Tata Motors_CV_ICV_FTU1         70        42          112
Tata Motors_CV_ICV_FTU2         48        71          119
Tata Motors_CV_ICV_RTLC1         0         5            5
Tata Motors_CV_ICV_SRT1         43        43           86
Tata Motors_CV_ICV_SRT2         14        13           27
Tata Motors_CV_LCV_FTU1         57        67          124
Tata Motors_CV_LCV_FTU2         21        32           53
Tata Motors_CV_LCV_RTLC1         3         4            7
Tata Motors_CV_LCV_RTLC2         7         7           14
Tata Motors_CV_LCV_SRT1         30        38           68
Tata Motors_CV_LCV_SRT2          4        11           15
Tata Motors_CV_LCV_SRT3          4         2            6
Tata Motors_CV_MHCV_FTU1       202       132          334
Tata Motors_CV_MHCV_FTU2        81       111          192
Tata Motors_CV_MHCV_RTLC1       17        30           47
Tata Motors_CV

In [12]:
df_new = df[['Product', 'Application Status']]

product_sampling = (
    df_new.groupby(['Product', 'Application Status'])
          .size()
          .unstack(fill_value=0)
)

product_sampling['Total'] = product_sampling.sum(axis=1)
product_sampling.loc['Total'] = product_sampling.sum()


print(product_sampling)


Application Status         Sampled  Screened  Total
Product                                            
Tata Motors_CV_ICV_FTU1         70        42    112
Tata Motors_CV_ICV_FTU2         48        71    119
Tata Motors_CV_ICV_RTLC1         0         5      5
Tata Motors_CV_ICV_SRT1         43        43     86
Tata Motors_CV_ICV_SRT2         14        13     27
Tata Motors_CV_LCV_FTU1         57        67    124
Tata Motors_CV_LCV_FTU2         21        32     53
Tata Motors_CV_LCV_RTLC1         3         4      7
Tata Motors_CV_LCV_RTLC2         7         7     14
Tata Motors_CV_LCV_SRT1         30        38     68
Tata Motors_CV_LCV_SRT2          4        11     15
Tata Motors_CV_LCV_SRT3          4         2      6
Tata Motors_CV_MHCV_FTU1       202       132    334
Tata Motors_CV_MHCV_FTU2        81       111    192
Tata Motors_CV_MHCV_RTLC1       17        30     47
Tata Motors_CV_MHCV_SRT1       105       154    259
Tata Motors_CV_MHCV_SRT2        30        40     70
Tata Motors_